In [ ]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name=name
        self.phone=phone
        self.email=email
        self.age=age
        self.grade=grade
        self.etc=etc
    def as_dic(self): #객체를 딕셔너리로 반환 (csv파일 저장시)
        return {'name':self.name, 'phone':self.phone, 'email':self.email, 'age':self.age, 'grade':self.grade, 'etc':self.etc}
    def as_list(self): #객체를 "홍길동 010-9999-9999 e@e.com 20 3 까칠"
        return [self.name, self.phone, self.email, self.age, self.grade, self.etc]
    def __str__(self): # *** 홍길동 010-9999-9999 e@e.com 20 까칠
        return "{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}".format('*'*self.grade, self.name, self.phone, self.email, self.age, self.etc)        

In [16]:
import cx_Oracle
oracle_dsn= cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
conn=cx_Oracle.connect(user="scott",password="tiger", dsn=oracle_dsn)
cursor=conn.cursor()

In [ ]:
try:
    cursor.execute("DROP TABLE MEMBER")
except:
    pass
cursor.execute("""
    CREATE TABLE MEMBER(
        name VARCHAR2(20),
        phone VARCHAR2(30),
        email VARCHAR2(100),
        age NUMBER(3),
        grade NUMBER(1),
        etc VARCHAR(100)
    )
""")

In [7]:
#1. 입력
def insert_member():
    cursor=conn.cursor()
    name=input("이름 : ")
    phone=input("전화번호 : ")
    email = input("이메일 : ")
    age=int(input("나이 : "))
    grade=int(input("고객등급(1~5) : "))
    etc=input("기타 정보 : ")
    member=Member(name,phone,email,age,grade,etc)
    cursor.execute("INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)",member.as_dic())
    conn.commit()
    cursor.close()
    pass

In [8]:
#2. 전체 출력
def select_all():
    import pandas as pd
    cursor=conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    #members=cursor.fetchall() #왜 cussor.fetchall()만 하면 안 되지????
    data=cursor.fetchall()
    data_df=pd.DataFrame(data)
    data_df.columns=[row[0] for row in cursor.description]
    print(data_df)
    cursor.close()
    pass

In [9]:
#3. 이름찾기
def select_name():
    cursor=conn.cursor()
    name=input("이름 : ")
    cursor.execute("SELECT * FROM MEMBER WHERE NAME = :name",{'name':name})
    members=cursor.fetchall() #왜 cussor.fetchall()만 하면 안 되지????
    for i in members:
        print(i)
    cursor.close()
    pass

In [10]:
#4. 메일 삭제 (했으면 했다,안했으면 안했다.)
def delete_member():
    cursor=conn.cursor()
    email=input("이메일 : ")
    cursor.execute("SELECT * FROM MEMBER WHERE EMAIL = :email",{'email':email})
    members=cursor.fetchall()
    if len(members)==0:
        print("그런 이메일 없다")
    else :
        print("해당 이메일 보유자 삭제 완료")
    cursor.execute("DELETE FROM MEMBER WHERE EMAIL= :email",{'email':email})
    conn.commit()
    cursor.close()
    pass

In [11]:
#5. 내보내기 (csv 헤더 포함)
def save_csv():
    import csv
    cursor=conn.cursor()
    filename=input("파일명 : ")
    members_list=[]
    cursor.execute("SELECT * FROM MEMBER")
    members=cursor.fetchall()
    fnames=[]
    for nm in [row[0] for row in cursor.description]:
        fnames.append(nm.lower())
    for i in members:
        member=Member(i[0],i[1],i[2],i[3],i[4],i[5])
        members_list.append(member)
    with open(filename,"w", encoding="utf-8", newline="") as f:
        dict_writer = csv.DictWriter(f, fieldnames=fnames)
        dict_writer.writeheader()
        for m in members_list:
            dict_writer.writerow(m.as_dic())
    cursor.close()
    pass

In [12]:
#0. 종료하기
def close_sql():
    conn.close()
    pass

In [13]:
def main():
    while True:
        print("1:입력|2:전체조회|3:이름찾기|4:메일삭제|5:내보내기(CSV)|0:종료")
        try:
            fn=int(input('메뉴 선택 : '))
        except Exception:
            print("숫자 1,2,3,4,5,0 중에 하나를 입력하시오")
            continue
        if fn==1:
            insert_member()
        elif fn==2:
            select_all()
        elif fn==3:
            select_name()
        elif fn==4:
            delete_member()
        elif fn==5:
            save_csv()
        elif fn==0:
            close_sql()
            break

In [23]:
if __name__=="__main__":
    import cx_Oracle
    oracle_dsn= cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
    global conn
    conn=cx_Oracle.connect(user="scott",password="tiger", dsn=oracle_dsn)
    main()

1:입력|2:전체조회|3:이름찾기|4:메일삭제|5:내보내기(CSV)|0:종료
메뉴 선택 : 0


In [24]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동','010-9999','emememem', :age, 3, '홍길동')",age=20)

In [25]:
cursor.execute("SELECT * FROM MEMBER")
cursor.fetchall()

[('홍길동', '010-9999', 'emememem', 20, 3, '홍길동'),
 ('3', '3', '3', 3, 3, '3'),
 ('홍길동', '010-9999', 'emememem', 20, 3, '홍길동'),
 ('홍길동', '010-9999', 'emememem', 20, 3, '홍길동'),
 ('홍길동', '010-9999', 'emememem', 20, 3, '홍길동')]

In [26]:
conn.commit()

InterfaceError: not connected